In [1]:
# Clone the repository
!git clone https://github.com/sathishkumar67/Browser-Automation.git

# move the files to the current directory
!mv /teamspace/studios/this_studio/Browser-Automation/* .

# upgrade pip
!pip install --upgrade pip

# install latest version pytorch
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

# install the required packages
!pip install -r requirements.txt

fatal: destination path 'Browser-Automation' already exists and is not an empty directory.


⚡️ Tip	Connect GitHub to Studios: https://lightning.ai/pt-sk/home?settings=integrations



zsh:1: no matches found: /teamspace/studios/this_studio/Browser-Automation/*
Looking in indexes: https://download.pytorch.org/whl/cu126


In [ ]:
# import the necessary libraries
from typing import Any
import torch
import gradio as gr
from PIL import Image
from transformers import AutoModelForImageTextToText, AutoProcessor
from transformers.models.qwen2_vl.image_processing_qwen2_vl import smart_resize


model_name = "Hcompany/Holo1.5-3B"  # or "Hcompany/Holo1.5-7B", "Hcompany/Holo1.5-72B"

# Load model and processor
model = AutoModelForImageTextToText.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_name)





def predict(image, task):
    # Resize image according to model's image processor
    image_processor_config = processor.image_processor
    resized_height, resized_width = smart_resize(
        image.height,
        image.width,
        factor=image_processor_config.patch_size * image_processor_config.merge_size,
        min_pixels=image_processor_config.min_pixels,
        max_pixels=image_processor_config.max_pixels,
    )

    processed_image: Image.Image = image.resize(size=(resized_width, resized_height), resample=Image.Resampling.LANCZOS)

    # Create the prompt
    messages: list[dict[str, Any]] = get_chat_messages(task, processed_image)

    # Apply chat template
    text_prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Process inputs
    inputs = processor(
        text=[text_prompt],
        images=[processed_image],
        padding=True,
        return_tensors="pt",
    ).to(model.device)

    # Generate response
    generated_ids = model.generate(**inputs, max_new_tokens=256)

    # Decode output
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    result = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return result



iface = gr.Interface(
    fn=predict,
    inputs=[gr.Image(type="pil"), gr.Textbox()],
    outputs=gr.JSON()
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860


* Running on public URL: https://5cf7a28d59bceee215.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
